In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 인공지능

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5

In [4]:
paper_link_list = []
#3
for i in range(3):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

FSM 기반의 게임 NPC 인공 지능 평가
이중외피 건물의 개구부 및 난방설비 제어를 위한 인공지능망의 적용
인공지능기법을 이용한 외환위기 조기경보시스템 구축
인공지능 기반 사람의 자세 분류를 이용한 두 바퀴 로봇의 움직임 제어
인공지능 : 스마트폰 가속도 센서 기반의 제스처 인식과 로봇 응용
인공지능을 이용한 파프리카 실내 양액 재배 시 발생하는 병해충 자동 검출
인공지능 : 스마트폰 가속도 센서를 이용한 행위 인식 시스템의 설계
인공지능 피난유도설비 적용에 따른 최적 대피시뮬레이션 연구
인공지능 : 제품수송 컨터네이너의 적재를 위한 규칙기반시스템
인공지능 : 자세 예측을 이용한 효과적인 자세 기반 감정 동작 인식
10
인공지능 : FMM 신경망에서 연관도요소를 이용한 규칙 추출 기법
인공지능 : 다중 분포 학습 모델을 위한 Haar-like Feature와 Decision Tree를 이용한 학습 알고리즘
인공지능 : S-100 표준 기반 피처 카탈로그 제작지원 시스템의 설계 및 구현
인공지능 : 영한 기계번역 시스템의 영한 변환사전 확장 도구
인공지능 : 다중 구간 샘플링에 기반한 배경제거 알고리즘
[인공지능, 신경망 및 퍼지시스템] 클라우드 컴퓨팅에 적합한 LDPC 부호 복제 기법
영어발음 향상을 위한 실시간 인공지능 입모양 인식 프로그램 개발
인공신경망을 이용한 지하채광 확정선외 혼입 예측과 분석 사례연구
지능을 이용한 온실 제어 시스템
도로기상정보시스템(RWIS)과 차량검지기(VDS) 자료를 이용한 강우수준별 통행속도예측
20
게임이론 관점에서 본 개인의 배반이 집단 이익에 미치는 영향
게임 캐릭터의 전술적 행동을 위한 위치 평가 함수의 학습
Particle Swarm Optimization을 이용한 공동주택의 공사비 예측에 관한 연구
인간의 인지능력 기반의 행동특성이 반영된 분산대피에 관한 연구
MCTS 기법을 활용한 불완전 정보 카드 게임에서의인공지능 에이전트 생성
사물인터넷을 접목한 반도체 소자 공정 최적화 시뮬레이터 개발
복수의 이미지를 합성하

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=93999d8ba3fd23fab7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=16b47975898671bbffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f9db2f034ad756bfffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=634c457452b0b7bfe9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=34111649c799be16e9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=93c707fba52fdd5ce9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a10b336ef610fc5fd18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=278c4617086494deffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=34111649c799be16d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

28

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=93999d8ba3fd23fab7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=16b47975898671bbffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f9db2f034ad756bfffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=634c457452b0b7bfe9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=34111649c799be16e9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=93c707fba52fdd5ce9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a10b336ef610fc5fd18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=278c4617086494deffe0bdc3ef48d419',
 'http://www.ris

In [9]:
len(paper_url)

28

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./6_인공지능.xlsx')
    
print("Running Time : ", time.time() - start)

0 ERROR
1 paper clear
2 paper clear
3 paper clear
4 ERROR
5 paper clear
6 ERROR
7 paper clear
8 ERROR
9 ERROR
10 ERROR
11 ERROR
12 ERROR
13 ERROR
14 ERROR
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 ERROR
25 ERROR
26 paper clear
27 paper clear
Running Time :  11.750559329986572
